In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.basics import *
import json

import jkbc.model as m
import jkbc.model.factory as factory
import jkbc.constants as constants
import jkbc.files.fasta as fasta
import jkbc.files.torch_files as f
import jkbc.utils.kd as kd
import jkbc.utils.preprocessing as prep
import jkbc.utils.postprocessing as pop

## Constants

### Data

In [16]:
BASE_DIR = Path("../..")
PATH_DATA = 'data/feather-files'
DATA_SET = 'Range0-10000-FixLabelLen400-winsize4096'
FEATHER_FOLDER = BASE_DIR/PATH_DATA/DATA_SET

with open(FEATHER_FOLDER/'config.json', 'r') as fp:
    config = json.load(fp)

ALPHABET       = constants.ALPHABET
ALPHABET_VAL   = list(ALPHABET.values())
ALPHABET_STR   = ''.join(ALPHABET_VAL)
ALPHABET_SIZE  = len(ALPHABET.keys())
WINDOW_SIZE    = int(config['maxw']) #maxw = max windowsize
DIMENSIONS_OUT = int(config['maxl']) # maxl = max label length
MIN_LABEL_LEN  = int(config['minl']) # maxl = max label length
STRIDE         = WINDOW_SIZE

### Train/Predict

In [17]:
BS = 2**3  # batch size
DEVICE = m.get_available_gpu() #torch.device("cpu")
DEVICE

device(type='cuda', index=1)

### Model

In [18]:
model, (MODEL_NAME, _) = factory.bonito(WINDOW_SIZE, DEVICE, 'quartznet5x5.toml')
MODEL_NAME = f'{MODEL_NAME}'
MODEL_DIR = f'{MODEL_NAME}/weights'
model_weights = 'bestmodel_8'
predicter = m.get_predicter(model, MODEL_DIR, DEVICE).to_fp16()

In [19]:
if not model_weights:
    model_weights = m.get_newest_model(MODEL_DIR)

predicter.load(model_weights);
model_weights

'bestmodel_8'

## Save teacher output

In [20]:
# Read data from feather
data = f.load_training_data(FEATHER_FOLDER)
kd.generate_and_save_y_teacher(FEATHER_FOLDER, MODEL_NAME, m.signal_to_input_tensor(data.x, DEVICE), predicter.model, bs=BS)

## Get accuracies

In [22]:
sc = prep.SignalCollection(BASE_DIR/constants.MAPPED_READS, BASE_DIR/constants.BACTERIA_DICT_PATH, labels_per_window=(MIN_LABEL_LEN,DIMENSIONS_OUT), 
                           stride=STRIDE, window_size=(WINDOW_SIZE-1, WINDOW_SIZE), blank_id=constants.BLANK_ID)

In [24]:
# predict range
predict_objects = m.predict_range(predicter.model, sc, ALPHABET_STR, WINDOW_SIZE, DEVICE, indexes=np.random.randint(0, len(sc), 10))




  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.60it/s]


 20%|██        | 2/10 [00:01<00:05,  1.55it/s]


 30%|███       | 3/10 [00:02<00:04,  1.53it/s]


 40%|████      | 4/10 [00:02<00:03,  1.79it/s]

Error: Skipping index 115407. No windows in signal





 60%|██████    | 6/10 [00:03<00:02,  1.94it/s]


 70%|███████   | 7/10 [00:04<00:01,  1.54it/s]


 80%|████████  | 8/10 [00:05<00:01,  1.26it/s]


 90%|█████████ | 9/10 [00:06<00:00,  1.21it/s]


100%|██████████| 10/10 [00:07<00:00,  1.35it/s]


In [ ]:
# convert outputs to dictionaries
references = []
predictions = []
accuracies = []
for po in predict_objects:
    [accuracies.append(a) for a in m.get_accuracies(po.references, po.predictions, ALPHABET_VAL)]
    ref, pred = fasta.map_decoded(po, ALPHABET_VAL, False)
    references.append(ref)
    predictions.append(pred)

# Sanity tjek
np.mean(accuracies)

In [ ]:
# make dicts ready to be saved
ref_dict = fasta.merge(references)
pred_dict = fasta.merge(predictions)

In [ ]:
# save dicts
fasta.save_dicts(pred_dict, ref_dict, f'{MODEL_NAME}/predictions/{DATA_SET}')